In [ ]:
import pandas as pd
import yfinance as yf
import sqlite3

In [3]:
conn = sqlite3.connect('finance.db')
stock_list = pd.read_sql_query("SELECT * FROM stock_list;", conn)
stock_data = pd.read_sql_query("SELECT * FROM stock_data;", conn)
conn.close()

In [5]:
stock_data

,Price,Ticker,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-08,2024-01-09,2024-01-10,2024-01-11,...,2025-07-10,2025-07-11,2025-07-14,2025-07-15,2025-07-16,2025-07-17,2025-07-18,2025-07-21,시장,종목명
0,Close,000020.KS,10193.579102,10251.883789,10203.296875,10174.144531,10300.471680,10310.188477,10319.906250,10203.296875,...,6890.0,6910.0,7010.0,6990.0,6930.0,7060.0,7000.0,7000.0,KOSPI,동화약품
1,Close,000040.KS,370.000000,367.000000,358.000000,357.000000,350.000000,354.000000,351.000000,355.000000,...,432.0,428.0,428.0,430.0,429.0,434.0,463.0,445.0,KOSPI,KR모터스
2,Close,000050.KS,8550.764648,8628.853516,8716.704102,8736.225586,8736.225586,8853.359375,8609.331055,8863.121094,...,8190.0,8150.0,8250.0,8320.0,8150.0,8230.0,8040.0,7870.0,KOSPI,경방
3,Close,000070.KS,64740.582031,63791.304688,63126.812500,63886.234375,62936.957031,62842.031250,62842.031250,63886.234375,...,99000.0,100000.0,100500.0,100600.0,98300.0,97100.0,94700.0,93100.0,KOSPI,삼양홀딩스
4,Close,000075.KS,49782.992188,48848.976562,49222.582031,49222.582031,49315.984375,48381.968750,49409.386719,49222.582031,...,96100.0,98000.0,98400.0,96500.0,97300.0,95600.0,94000.0,92900.0,KOSPI,삼양홀딩스우
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13775,Volume,950170.KQ,26285.000000,23355.000000,20630.000000,23050.000000,18164.000000,16985.000000,18914.000000,20175.000000,...,259712.0,195243.0,207245.0,256919.0,210392.0,102800.0,626637.0,483876.0,KOSDAQ,JTC
13776,Volume,950190.KQ,51260.000000,24548.000000,17404.000000,20745.000000,19906.000000,58841.000000,54335.000000,25460.000000,...,57589.0,106056.0,80828.0,145084.0,64823.0,67281.0,29249.0,21976.0,KOSDAQ,고스트스튜디오
13777,Volume,950200.KQ,16905.000000,68626.000000,41592.000000,15401.000000,19993.000000,700384.000000,106566.000000,72209.000000,...,58618.0,34021.0,21897.0,14315.0,34736.0,36273.0,12304.0,12712.0,KOSDAQ,소마젠
13778,Volume,950210.KS,60857.000000,64322.000000,36823.000000,36109.000000,25942.000000,25583.000000,43320.000000,41975.000000,...,442200.0,477717.0,483376.0,603081.0,234793.0,195341.0,251511.0,282924.0,KOSPI,프레스티지바이오파마


In [6]:
short_window = 20
long_window = 60
start_date = '2024-06-01'
end_date = '2025-06-30'

# 날짜 컬럼만 추출
date_cols = stock_data.columns[2:-2]
date_cols = pd.to_datetime(date_cols)
target_dates = [d.strftime('%Y-%m-%d') for d in date_cols if start_date <= d.strftime('%Y-%m-%d') <= end_date]

# 패션플랫폼의 Close row만 추출
row = stock_data[(stock_data['종목명'] == '패션플랫폼') & (stock_data['Price'] == 'Close')].iloc[0]
price_series = row[2:-2]
price_series.index = pd.to_datetime(price_series.index)
price_series = price_series.astype(float).sort_index()

# 이동평균 계산
ma_short = price_series.rolling(window=short_window, min_periods=1).mean()
ma_long = price_series.rolling(window=long_window, min_periods=1).mean()

# 차이 계산
signal = ma_short - ma_long
signal = signal.loc[start_date:end_date]

# 신호 변화 (양수→음수: 데드크로스, 음수→양수: 골든크로스)
cross = signal.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
diff = cross.diff()

golden_crosses = (diff == 2).sum()
dead_crosses = (diff == -2).sum()

print(f"골든크로스: {golden_crosses}회, 데드크로스: {dead_crosses}회")

골든크로스: 3회, 데드크로스: 3회


In [8]:
ticker = stock_list[stock_list['종목명'] == '대성미생물']['yfinance_ticker'].values[0]
price = stock_data[(stock_data['Ticker'] == ticker) & (stock_data['Price'] == 'Close')]['2024-10-28'].values[0]
print(f"{int(round(price))}원")   


10008원


In [ ]:
# 파라미터 설정
target_date = '2024-08-02'
rolling_window = 20
threshold = 5  # 500% 이상

# 날짜 컬럼만 추출
date_cols = stock_data.columns[2:-2]

# 거래량 데이터만 필터링
volume_data = stock_data[stock_data['Price'] == 'Volume'].copy()
volume_only = volume_data[date_cols].astype(float)

# 20일 이동평균 계산
ma20 = volume_only.T.rolling(window=rolling_window, min_periods=rolling_window).mean().T

# 거래량 및 이동평균 비교
volume_data['거래량'] = volume_data[target_date]
volume_data['MA20'] = ma20[target_date]

# 거래량이 이동평균의 5배 이상인 종목 필터링
filtered = volume_data[volume_data['거래량'] >= volume_data['MA20'] * threshold].copy()

# 상승률 계산 및 정렬
filtered['상승률'] = ((filtered['거래량'] / filtered['MA20']) - 1) * 100
filtered = filtered.sort_values(by='상승률', ascending=False)

# 결과 문자열 포맷팅
result = filtered.apply(
    lambda row: f"{row['종목명']}({row['상승률']:.2f}%)", axis=1
)

# 출력
print(", ".join(result))

대성하이텍(1624.70%), 휴니드(1604.07%), 소노스퀘어(1532.09%), 진매트릭스(1480.57%), 에스티오(1473.64%), DGP(1307.24%), 바이오노트(1297.70%), 코데즈컴바인(1292.21%), 나노엔텍(1246.12%), 미래생명자원(1200.96%), 랩지노믹스(1134.88%), 블루콤(1099.83%), 수젠텍(983.85%), 씨젠(965.09%), 피씨엘(850.23%), 휴마시스(821.86%), 엑세스바이오(782.72%), 한일사료(776.32%), 엘컴텍(769.23%), 형지글로벌(754.31%), 스페코(743.35%), 더바이오메드(723.52%), 파루(676.44%), 퀄리타스반도체(643.03%), 셀리드(640.79%), 멕아이씨에스(635.53%), 흥아해운(629.97%), SBI핀테크솔루션즈(600.16%), 한국석유(594.40%), 대주산업(588.17%), 오브젠(552.73%), 중앙에너비스(546.06%), 빅텍(508.55%), 유투바이오(484.44%), 평화홀딩스(466.84%), 에스디바이오센서(463.63%), 중앙백신(461.58%), 하츠(458.63%), 고려산업(458.22%), 케어젠(451.40%), 흥구석유(451.30%), 알리코제약(440.56%), GS리테일(410.94%), 국제약품(410.62%), 우원개발(400.83%), 대신밸런스제16호스팩(400.21%), DH오토넥스(nan%), 만호제강(nan%), 국보(nan%), IHQ(nan%), 부산주공(nan%), 한창(nan%), 카프로(nan%), 선도전기(nan%), 대동전자(nan%), 태영건설(nan%), 태영건설우(nan%), 웰바이오텍(nan%), 광림(nan%), 현대사료(nan%), 에스유앤피(nan%), 이화전기(nan%), 더라미(nan%), KH 필룩스(nan%), 코맥스(nan%), 플래스크(nan%), 조광ILI(nan%), 코다코(nan%), 초록뱀미디어(nan%),

In [13]:
# 종목명 : 대성미생물
# 날짜 : 2024-10-28

import pandas as pd

ticker_name = "대성미생물"
target_date = "2024-10-28"

date_cols = [c for c in stock_data.columns if c not in ['Price','Ticker','시장','종목명']]
close_data = stock_data[(stock_data['Price'] == 'Close') & (stock_data['종목명'] == ticker_name) & (stock_data['시장'] == 'KOSDAQ')].copy()
close_only = close_data[date_cols].astype(float)

price = close_only[target_date].iloc[0]
print(f"{price:,.0f}원")

10,008원


In [28]:
import pandas as pd


target_date = "2024-01-31"

date_cols = [c for c in stock_data.columns if c not in ['Price','Ticker','시장','종목명']]
value_data = stock_data[stock_data['Price'] == 'Volume'].copy()
value_only = value_data[date_cols].astype(float)

total_value = value_only[target_date].sum()
print(f"{int(total_value):,}원")

1,500,342,726원


In [34]:
value_data

,Price,Ticker,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-08,2024-01-09,2024-01-10,2024-01-11,...,2025-07-10,2025-07-11,2025-07-14,2025-07-15,2025-07-16,2025-07-17,2025-07-18,2025-07-21,시장,종목명
11024,Volume,000020.KS,238593.0,120979.0,155776.0,132981.0,129013.0,129810.0,67069.0,81754.0,...,90194.0,133211.0,62178.0,58905.0,43590.0,107095.0,54442.0,77829.0,KOSPI,동화약품
11025,Volume,000040.KS,636984.0,158370.0,237942.0,138829.0,464629.0,189347.0,226064.0,173488.0,...,171115.0,126470.0,119835.0,165422.0,149666.0,183150.0,646599.0,264709.0,KOSPI,KR모터스
11026,Volume,000050.KS,3284.0,5933.0,4808.0,3868.0,4365.0,9412.0,7412.0,6908.0,...,121734.0,77947.0,46740.0,133853.0,52881.0,55783.0,37282.0,72941.0,KOSPI,경방
11027,Volume,000070.KS,12600.0,15465.0,13330.0,18887.0,8558.0,5120.0,7960.0,21276.0,...,67092.0,45862.0,31844.0,25152.0,28955.0,26074.0,29720.0,27309.0,KOSPI,삼양홀딩스
11028,Volume,000075.KS,112.0,169.0,36.0,367.0,228.0,88.0,242.0,74.0,...,386.0,452.0,114.0,505.0,27.0,147.0,164.0,37.0,KOSPI,삼양홀딩스우
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13775,Volume,950170.KQ,26285.0,23355.0,20630.0,23050.0,18164.0,16985.0,18914.0,20175.0,...,259712.0,195243.0,207245.0,256919.0,210392.0,102800.0,626637.0,483876.0,KOSDAQ,JTC
13776,Volume,950190.KQ,51260.0,24548.0,17404.0,20745.0,19906.0,58841.0,54335.0,25460.0,...,57589.0,106056.0,80828.0,145084.0,64823.0,67281.0,29249.0,21976.0,KOSDAQ,고스트스튜디오
13777,Volume,950200.KQ,16905.0,68626.0,41592.0,15401.0,19993.0,700384.0,106566.0,72209.0,...,58618.0,34021.0,21897.0,14315.0,34736.0,36273.0,12304.0,12712.0,KOSDAQ,소마젠
13778,Volume,950210.KS,60857.0,64322.0,36823.0,36109.0,25942.0,25583.0,43320.0,41975.0,...,442200.0,477717.0,483376.0,603081.0,234793.0,195341.0,251511.0,282924.0,KOSPI,프레스티지바이오파마
